In [83]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore')

In [84]:
positions = pd.read_csv("portfolio_positions_exploded.csv")
costs = pd.read_csv("portfolio_costs_exploded.csv")

Construção do PNL a nível do ativo

In [85]:
positions['pct_exposicao'] = positions['exposure_value'] / positions['portfolio_nav']
positions['dtd_ativo'] = positions['dtd_chg'] / positions['pct_exposicao']

positions['overview_date'] = pd.to_datetime(positions['overview_date'])
positions = positions.sort_values(['portfolio_id','instrument_name','portfolio_origem','overview_date'])

positions['Year'] = positions['overview_date'].dt.year
positions['Month'] = positions['overview_date'].dt.month

# Calcula ano/mês do mês anterior
positions['Month_after'] = positions['Month'] + 1
positions['Year_after'] = positions['Year']
positions.loc[positions['Month_after'] == 13, 'Month_after'] = 1
positions.loc[positions['Month'] == 12, 'Year_after'] += 1

# Tabela com último exposure_value do mês para cada grupo
last_prev_month = (
    positions
    .groupby(['portfolio_id','instrument_name','portfolio_origem','Year','Month'])
    .apply(lambda g: g.loc[g['overview_date'] == g['overview_date'].max()])
    .reset_index(drop=True)
)[['portfolio_id','instrument_name','portfolio_origem','Year_after','Month_after','exposure_value']]

last_prev_month = last_prev_month.rename(
    columns={'Year_after':'Year','Month_after':'Month','exposure_value':'exposure_value_prev'}
)

# Faz merge para trazer o exposure_value_prev para cada linha
positions = positions.merge(
    last_prev_month,
    on=['portfolio_id','instrument_name','portfolio_origem','Year','Month'],
    how='left'
)

# Calcula o PnL MTD pra qualquer dia:
positions['mtd_ativo'] = (positions['exposure_value'] - positions['exposure_value_prev']) / positions['exposure_value_prev']
positions['mtd_financeiro'] = positions['exposure_value'] - positions['exposure_value_prev']



Construção do PNL da carteira

Positions MTD

In [96]:
# PnL MTD
positions['pnl_mtd'] = positions.groupby(['portfolio_id','instrument_name','portfolio_origem', 'Year', 'Month'])['dtd_chg'].transform(
    lambda x: ( (1 + x).cumprod() - 1 )
)

group_cols = ['portfolio_id','instrument_name','portfolio_origem','Year','Month']
all_dates = positions.groupby(group_cols)['overview_date'].agg(['min', 'max']).reset_index()

# Gera linhas para todos os dias do mês para cada ativo
expanded = []
for _, row in all_dates.iterrows():
    year = row['Year']
    month = row['Month']
    filtro = (positions['overview_date'].dt.year == year) & (positions['overview_date'].dt.month == month)

    # pega o último dia do mês de forma automática
    dates = list(positions['overview_date'][filtro].drop_duplicates())
    for date in dates:
        expanded.append({**row, 'overview_date': date})

calendar = pd.DataFrame(expanded)

# Passo 2: Merge calendar com positions
positions_full = pd.merge(calendar, positions, on=group_cols + ['overview_date'], how='left')

# Passo 3: Forward fill do PnL e das outras colunas por ativo/mês
positions_full = positions_full.sort_values(group_cols + ['overview_date'])
positions_full['pnl_mtd'] = positions_full.groupby(group_cols)['pnl_mtd'].ffill()

In [87]:
costs['overview_date'] = pd.to_datetime(costs['overview_date'])
costs['Year'] = costs['overview_date'].dt.year
costs['Month'] = costs['overview_date'].dt.month

costs = costs.sort_values(['origin_portfolio_id', 'overview_date'])

# PnL MTD
costs['pnl_mtd'] = costs.groupby(['category_name','origin_portfolio_id', 'root_portfolio', 'Year', 'Month'])['dtd_chg'].transform(
    lambda x: ( (1 + x).cumprod() - 1)
)

costs['pnl_ytd'] = costs.groupby(['category_name','origin_portfolio_id', 'root_portfolio', 'Year'])['dtd_chg'].transform(
    lambda x: ( (1 + x).cumprod() - 1)
)



In [ ]:
positions_full.drop(columns=['Month','Year','Month_after','Year_after','exposure_value_prev','min','max'],inplace=True)
positions_full.replace([np.inf, -np.inf], 0, inplace=True)
costs.replace([np.inf, -np.inf], 0, inplace=True)

positions_full.to_csv("portfolio_positions_exploded.csv", index=False)
costs.to_csv("portfolio_costs_exploded.csv", index=False)

In [91]:
df_filtrado = positions_full[
    (positions_full['portfolio_id'] == 1175)
]

df_filtrado.to_clipboard()

In [90]:
df_filtrado = costs[
    (costs['root_portfolio'] == 1247)
]

df_filtrado.to_clipboard()